```
[XXX@cirrus-login0 lab_exercises]$ pip install nltk
[XXX@cirrus-login0 lab_exercises]$ python
Python 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import nltk
>>> nltk.download('wordnet')
>>> nltk.download('punkt')
>>> nltk.download('stopwords')
```

In [70]:
import nltk
import string

This is the format of the CSV file:
title,edition,year,place,archive_filename,page_filename,page_id,num_pages,type_archive,model,preprocess,page_string

In [79]:
df= sqlContext.read.csv("/lustre/home/shared/y15/spark/data/nls_demo.csv", header="true")
df.show(3)

+--------------------+--------------------+----+---------+--------------------+--------------------+-------+---------+------------+-----+--------------------+--------------------+
|               title|             edition|year|    place|    archive_filename|       page_filename|page_id|num_pages|type_archive|model|          preprocess|         page_string|
+--------------------+--------------------+----+---------+--------------------+--------------------+-------+---------+------------+-----+--------------------+--------------------+
|Encyclopaedia Bri...|Seventh edition, ...|1842|Edinburgh|/mnt/lustre/at003...|alto/193201290.34...|  Page1|      810|        book|  nls|PreprocessWordTyp...|                null|
|Encyclopaedia Bri...|Seventh edition, ...|1842|Edinburgh|/mnt/lustre/at003...|alto/193201303.34...|  Page2|      810|        book|  nls|PreprocessWordTyp...|                null|
|Encyclopaedia Bri...|Seventh edition, ...|1842|Edinburgh|/mnt/lustre/at003...|alto/193201316.34...|

In [80]:
# Filter out the pages that are null, and select only 3 columns.
newdf=df.filter(df.page_string.isNotNull()).select(df.year, df.page_string)
newdf.printSchema()

root
 |-- year: string (nullable = true)
 |-- page_string: string (nullable = true)



In [81]:
newdf.count()

14

In [82]:
newdf.show()

+----+--------------------+
|year|         page_string|
+----+--------------------+
|1842|EG.nr • «- .V % ^...|
|1842|          ■ t ■f 4 %|
|1842|                   /|
|1842|ENCYCLOPAEDIA BRI...|
|1842|THE ENCYCLOPAEDIA...|
|1842|ENCYCLOPAEDIA BRI...|
|1842|2 LAB I>abarum ed...|
|1842|LAB Laboureur whe...|
|1842|4 LAB L A B Labra...|
|1842|5 LAB Labrum men ...|
|1842|6 LAC Lace, about...|
|1842|7 LAC Lace, time ...|
|1842|8 LAC Laced;*;- t...|
|1842|LAC Lactantius. l...|
+----+--------------------+



In [83]:
# Converting the dataframe to tuples
pages=newdf.rdd.map(tuple)

In [85]:
pages.take(8)

[('1842', 'EG.nr • «- .V % ^ 1 \\ * •V'),
 ('1842', '■ t ■f 4 %'),
 ('1842', '/'),
 ('1842', 'ENCYCLOPAEDIA BRITANNICA SEVENTH EDITION.'),
 ('1842',
  'THE ENCYCLOPAEDIA BRITANNICA DICTIONARY OF ARTS, SCIENCES, AND GENERAL LITERATURE. SEVENTH EDITION, i WITH PRELIMINARY DISSERTATIONS ON THE HISTORY OF THE SCIENCES, AND OTHER EXTENSIVE IMPROVEMENTS AND ADDITIONS; INCLUDING THE LATE SUPPLEMENT. A GENERAL INDEX, AND NUMEROUS ENGRAVINGS. VOLUME XIII. ADAM AND CHARLES BLACK, EDINBURGH; M.DCCC.XLII.'),
 ('1842',
  "ENCYCLOPAEDIA BRITANNICA L. LAB A SEMI-YO^VEL, or liquid, forming the eleventh 9 letter of the alphabet. It is derived from the old ,abadie. Hebrew lamed, or Greek lambda X. It is sounded by inter- —v - —''cepting the breath between the tip of the tongue and the fore part of the palate, with the mouth open, and makes a sweet sound, with something of an aspiration ; hence the Britons and Spaniards usually doubled it, or added an h to it, in the beginning of words, as in Man, or lha

In [86]:
def sent_TokenizeFunct(x):
    print ("%s" %x)
    return nltk.sent_tokenize(x)

In [87]:
sentenceTokenizeRDD = pages.map(lambda p: sent_TokenizeFunct(p[1]))

In [88]:
sentenceTokenizeRDD.take(5)

[['EG.nr • «- .V % ^ 1 \\ * •V'],
 ['■ t ■f 4 %'],
 ['/'],
 ['ENCYCLOPAEDIA BRITANNICA SEVENTH EDITION.'],
 ['THE ENCYCLOPAEDIA BRITANNICA DICTIONARY OF ARTS, SCIENCES, AND GENERAL LITERATURE.',
  'SEVENTH EDITION, i WITH PRELIMINARY DISSERTATIONS ON THE HISTORY OF THE SCIENCES, AND OTHER EXTENSIVE IMPROVEMENTS AND ADDITIONS; INCLUDING THE LATE SUPPLEMENT.',
  'A GENERAL INDEX, AND NUMEROUS ENGRAVINGS.',
  'VOLUME XIII.',
  'ADAM AND CHARLES BLACK, EDINBURGH; M.DCCC.XLII.']]

In [89]:
def word_TokenizeFunct(x):
    splitted = [word for line in x for word in line.split()]
    return splitted

In [90]:
wordTokenizeRDD = sentenceTokenizeRDD.map(word_TokenizeFunct)

In [92]:
wordTokenizeRDD.take(5)

[['EG.nr', '•', '«-', '.V', '%', '^', '1', '\\', '*', '•V'],
 ['■', 't', '■f', '4', '%'],
 ['/'],
 ['ENCYCLOPAEDIA', 'BRITANNICA', 'SEVENTH', 'EDITION.'],
 ['THE',
  'ENCYCLOPAEDIA',
  'BRITANNICA',
  'DICTIONARY',
  'OF',
  'ARTS,',
  'SCIENCES,',
  'AND',
  'GENERAL',
  'LITERATURE.',
  'SEVENTH',
  'EDITION,',
  'i',
  'WITH',
  'PRELIMINARY',
  'DISSERTATIONS',
  'ON',
  'THE',
  'HISTORY',
  'OF',
  'THE',
  'SCIENCES,',
  'AND',
  'OTHER',
  'EXTENSIVE',
  'IMPROVEMENTS',
  'AND',
  'ADDITIONS;',
  'INCLUDING',
  'THE',
  'LATE',
  'SUPPLEMENT.',
  'A',
  'GENERAL',
  'INDEX,',
  'AND',
  'NUMEROUS',
  'ENGRAVINGS.',
  'VOLUME',
  'XIII.',
  'ADAM',
  'AND',
  'CHARLES',
  'BLACK,',
  'EDINBURGH;',
  'M.DCCC.XLII.']]

In [1]:
def removeStopWordsFunct(x):
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

stopwordRDD = wordTokenizeRDD.map(removeStopWordsFunct)

In [95]:
stopwordRDD.take(10)

[['EG.nr', '•', '«-', '.V', '%', '^', '1', '\\', '*', '•V'],
 ['■', '■f', '4', '%'],
 ['/'],
 ['ENCYCLOPAEDIA', 'BRITANNICA', 'SEVENTH', 'EDITION.'],
 ['THE',
  'ENCYCLOPAEDIA',
  'BRITANNICA',
  'DICTIONARY',
  'OF',
  'ARTS,',
  'SCIENCES,',
  'AND',
  'GENERAL',
  'LITERATURE.',
  'SEVENTH',
  'EDITION,',
  'WITH',
  'PRELIMINARY',
  'DISSERTATIONS',
  'ON',
  'THE',
  'HISTORY',
  'OF',
  'THE',
  'SCIENCES,',
  'AND',
  'OTHER',
  'EXTENSIVE',
  'IMPROVEMENTS',
  'AND',
  'ADDITIONS;',
  'INCLUDING',
  'THE',
  'LATE',
  'SUPPLEMENT.',
  'A',
  'GENERAL',
  'INDEX,',
  'AND',
  'NUMEROUS',
  'ENGRAVINGS.',
  'VOLUME',
  'XIII.',
  'ADAM',
  'AND',
  'CHARLES',
  'BLACK,',
  'EDINBURGH;',
  'M.DCCC.XLII.'],
 ['ENCYCLOPAEDIA',
  'BRITANNICA',
  'L.',
  'LAB',
  'A',
  'SEMI-YO^VEL,',
  'liquid,',
  'forming',
  'eleventh',
  '9',
  'letter',
  'alphabet.',
  'It',
  'derived',
  'old',
  ',abadie.',
  'Hebrew',
  'lamed,',
  'Greek',
  'lambda',
  'X.',
  'It',
  'sounded',
  'inter

In [96]:
def removePunctuationsFunct(x):
    nltk.download('punkt')
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered

In [97]:
rmvPunctRDD = stopwordRDD.map(removePunctuationsFunct)

In [98]:
rmvPunctRDD.take(10)

[['EGnr', '•', '«', 'V', '', '', '1', '', '', '•V'],
 ['■', '■f', '4', ''],
 [''],
 ['ENCYCLOPAEDIA', 'BRITANNICA', 'SEVENTH', 'EDITION'],
 ['THE',
  'ENCYCLOPAEDIA',
  'BRITANNICA',
  'DICTIONARY',
  'OF',
  'ARTS',
  'SCIENCES',
  'AND',
  'GENERAL',
  'LITERATURE',
  'SEVENTH',
  'EDITION',
  'WITH',
  'PRELIMINARY',
  'DISSERTATIONS',
  'ON',
  'THE',
  'HISTORY',
  'OF',
  'THE',
  'SCIENCES',
  'AND',
  'OTHER',
  'EXTENSIVE',
  'IMPROVEMENTS',
  'AND',
  'ADDITIONS',
  'INCLUDING',
  'THE',
  'LATE',
  'SUPPLEMENT',
  'A',
  'GENERAL',
  'INDEX',
  'AND',
  'NUMEROUS',
  'ENGRAVINGS',
  'VOLUME',
  'XIII',
  'ADAM',
  'AND',
  'CHARLES',
  'BLACK',
  'EDINBURGH',
  'MDCCCXLII'],
 ['ENCYCLOPAEDIA',
  'BRITANNICA',
  'L',
  'LAB',
  'A',
  'SEMIYOVEL',
  'liquid',
  'forming',
  'eleventh',
  '9',
  'letter',
  'alphabet',
  'It',
  'derived',
  'old',
  'abadie',
  'Hebrew',
  'lamed',
  'Greek',
  'lambda',
  'X',
  'It',
  'sounded',
  'inter',
  '—v',
  '',
  '—cepting',
  'br

In [99]:
def lemmatizationFunct(x):
    nltk.download('wordnet')
    lemmatizer = nltk.WordNetLemmatizer()
    finalLem = [lemmatizer.lemmatize(s) for s in x]
    return finalLem

In [100]:
lem_wordsRDD = rmvPunctRDD.map(lemmatizationFunct)

In [101]:
lem_wordsRDD.take(10)

[['EGnr', '•', '«', 'V', '', '', '1', '', '', '•V'],
 ['■', '■f', '4', ''],
 [''],
 ['ENCYCLOPAEDIA', 'BRITANNICA', 'SEVENTH', 'EDITION'],
 ['THE',
  'ENCYCLOPAEDIA',
  'BRITANNICA',
  'DICTIONARY',
  'OF',
  'ARTS',
  'SCIENCES',
  'AND',
  'GENERAL',
  'LITERATURE',
  'SEVENTH',
  'EDITION',
  'WITH',
  'PRELIMINARY',
  'DISSERTATIONS',
  'ON',
  'THE',
  'HISTORY',
  'OF',
  'THE',
  'SCIENCES',
  'AND',
  'OTHER',
  'EXTENSIVE',
  'IMPROVEMENTS',
  'AND',
  'ADDITIONS',
  'INCLUDING',
  'THE',
  'LATE',
  'SUPPLEMENT',
  'A',
  'GENERAL',
  'INDEX',
  'AND',
  'NUMEROUS',
  'ENGRAVINGS',
  'VOLUME',
  'XIII',
  'ADAM',
  'AND',
  'CHARLES',
  'BLACK',
  'EDINBURGH',
  'MDCCCXLII'],
 ['ENCYCLOPAEDIA',
  'BRITANNICA',
  'L',
  'LAB',
  'A',
  'SEMIYOVEL',
  'liquid',
  'forming',
  'eleventh',
  '9',
  'letter',
  'alphabet',
  'It',
  'derived',
  'old',
  'abadie',
  'Hebrew',
  'lamed',
  'Greek',
  'lambda',
  'X',
  'It',
  'sounded',
  'inter',
  '—v',
  '',
  '—cepting',
  'br